Importing necessary libraries numpy and pandas

In [10]:

import numpy as np
import pandas as pd


Reading CSV files using pandas and storing them in separate dataframes

In [11]:
df1 = pd.read_csv('src/ratings/1.csv')
df2 = pd.read_csv('src/ratings/2.csv')
df3 = pd.read_csv('src/ratings/3.csv')
df4 = pd.read_csv('src/ratings/4.csv')
df5 = pd.read_csv('src/ratings/5.csv')
df6 = pd.read_csv('src/ratings/6.csv')
df7 = pd.read_csv('src/ratings/7.csv')
df8 = pd.read_csv('src/ratings/8.csv')

Concatenating all the dataframes into a single dataframe using pandas concat function

In [12]:
dfratings=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8],ignore_index=True)


Renaming the 'rating' column to 'score' using the rename() function in pandas

In [13]:
dfratings = dfratings.rename(columns={'rating': 'score'})


Renaming the 'movieId' column to 'id' using the rename() function in pandas

In [14]:
dfratings = dfratings.rename(columns={'movieId': 'id'})

show data frame 

In [ ]:
dfratings

converting the dataframe to parquet format and saving to disk

In [16]:
import pyarrow

dfratings.to_parquet('dfrating.parquet')

In [17]:
dfprt= pd.read_parquet('dfrating.parquet')

In [ ]:
dfprt

Reading CSV files into separate dataframes for Amazon Prime, Disney Plus, Hulu, and Netflix

In [19]:
#se hace la carga de los archivos csv de las plataformas de streaming
dfa = pd.read_csv('src/amazon_prime_titles.csv', index_col=False)

dfd = pd.read_csv('src/disney_plus_titles.csv', index_col=False)

dfh = pd.read_csv('src/hulu_titles.csv', index_col=False)

dfn = pd.read_csv('src/netflix_titles.csv', index_col=False)

Adding a 'platform' column to each dataframe to identify the source platform

In [20]:
#creamos una columna de plataforma que contenga como valor la primera letra de cada plataforma de streaming en cada uno de los df

dfn['plataforma']= 'n'

dfa['plataforma']= 'a'

dfd['plataforma']= 'd'

dfh['plataforma']= 'h'

Concatenating all the dataframes of different platforms into a single dataframe using pandas concat function

In [21]:
dfplataformas = pd.concat([dfa,dfn,dfh,dfd],ignore_index=True)

Adding a new column 'id' to the combined dataframe that consists of the platform and show_id

In [22]:
dfplataformas["id"] = dfplataformas["plataforma"] + dfplataformas["show_id"].astype(str)


Converting the 'date_added' column from string to datetime format using the pandas to_datetime() function

In [23]:
dfplataformas["date_added"] = pd.to_datetime(dfplataformas["date_added"]).dt.strftime("%Y-%m-%d")


Replacing all missing values in the 'rating' column with the string value "G" using the fillna() function

In [24]:
dfplataformas["rating"] = dfplataformas["rating"].fillna("G")


Using the applymap() function to apply a lambda function to all elements of the dataframe
The lambda function checks if the element is a string, and if so, converts it to lowercase

In [25]:
dfplataformas = dfplataformas.applymap(lambda x: x.lower() if type(x) == str else x)


Splitting the 'duration' column into two separate columns using the pandas str.split() function
The 'duration_int' column will contain the numerical part of the duration and 'duration_type' column will contain the unit of time (e.g. minutes, seasons)

In [26]:
dfplataformas[['duration_int', 'duration_type']] = dfplataformas['duration'].str.split(' ',1,expand=True)

C:\Users\luis-\AppData\Local\Temp\ipykernel_3440\2105690346.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dfplataformas[['duration_int', 'duration_type']] = dfplataformas['duration'].str.split(' ',1,expand=True)


función personalizada para calcular el promedio y redondearlo
calcular los promedios redondeados y crear el nuevo DataFrame

In [27]:
# función personalizada para calcular el promedio y redondearlo
def promedio_redondeado(x):
    return round(x.mean(), 2)

# calcular los promedios redondeados y crear el nuevo DataFrame
datos = dfratings.groupby('id').agg({'score': promedio_redondeado}).reset_index()
datos.columns = ['id', 'ScoreMedio']

Merging the 'dfplataformas' dataframe with the 'datos' dataframe on the 'id' column using the pandas merge() function
This creates a new dataframe 'datos_finales' that contains all the columns from both dataframes and is merged on the common 'id' column

In [28]:

datos_finales = pd.merge(dfplataformas, datos, on='id')

Exporting the final dataframe 'datos_finales' to a csv file using the to_csv() function
The csv file is saved as 'df_plataformas_final.csv' in the current working directory

In [29]:
datos_finales.to_csv('df_plataformas_final.csv')

Merging the 'dfratings' dataframe with the 'dfplataformas' dataframe on the 'id' column using the pandas merge() function
This creates a new dataframe 'df_final' that contains all the columns from both dataframes and is merged on the common 'id' column

In [31]:
df_final= pd.merge(dfratings, dfplataformas, on='id')

Exporting the final dataframe 'df_final' to a Parquet file using the to_parquet() function
The Parquet file is saved as 'df_final.parquet' in the current working directory

In [32]:
df_final.to_parquet('df_final.parquet')